In [ ]:
{
 "cells": [
  {
   "cell_type": "code",
   "execution_count": 0,
   "metadata": {
    "application/vnd.databricks.v1+cell": {
     "cellMetadata": {
      "byteLimit": 2048000,
      "implicitDf": true,
      "rowLimit": 10000
     },
     "inputWidgets": {},
     "nuid": "d610686f-0a21-4ef8-8b44-73fbd1f2e3a3",
     "showTitle": false,
     "tableResultSettingsMap": {},
     "title": ""
    }
   },
   "outputs": [],
   "source": [
    "%sql\n",
    "-- 1. Bảng RAW: orders_raw (streaming từ parquet)\n",
    "\n",
    "CREATE OR REFRESH STREAMING LIVE TABLE orders_raw\n",
    "COMMENT \"The raw orders data, ingested from parquet files.\"\n",
    "AS SELECT * \n",
    "FROM cloud_files(\"${datasets.path}/orders\", \"parquet\", map(\"cloudFiles.inferColumnTypes\", \"true\"));\n",
    "\n",
    "-- 2. Bảng RAW: customers (streaming từ JSON)\n",
    "\n",
    "CREATE OR REFRESH STREAMING LIVE TABLE customers_raw\n",
    "COMMENT \"The raw customers data, ingested from JSON files.\"\n",
    "AS SELECT * \n",
    "FROM cloud_files(\"${datasets.path}/customers\", \"json\", map(\"cloudFiles.inferColumnTypes\", \"true\"));\n",
    "\n",
    "-- 3. Bảng cleaned: orders_cleaned\n",
    "\n",
    "CREATE OR REFRESH LIVE TABLE orders_cleaned\n",
    "COMMENT \"Cleaned orders data with non-null order_id and positive quantity\"\n",
    "AS SELECT * \n",
    "FROM LIVE.orders_raw\n",
    "WHERE order_id IS NOT NULL AND quantity > 0;\n",
    "\n",
    "-- 4. Bảng final: enriched_orders\n",
    "\n",
    "CREATE OR REFRESH LIVE TABLE enriched_orders\n",
    "COMMENT \"Join orders with customer data\"\n",
    "AS SELECT \n",
    "  o.order_id,\n",
    "  o.customer_id,\n",
    "  c.name AS customer_name,\n",
    "  o.quantity,\n",
    "  o.price,\n",
    "  o.order_timestamp\n",
    "FROM LIVE.orders_cleaned o\n",
    "LEFT JOIN LIVE.customers_raw c\n",
    "ON o.customer_id = c.customer_id;\n"
   ]
  }
 ],
 "metadata": {
  "application/vnd.databricks.v1+notebook": {
   "computePreferences": null,
   "dashboards": [],
   "environmentMetadata": {
    "base_environment": "",
    "environment_version": "2"
   },
   "inputWidgetPreferences": null,
   "language": "python",
   "notebookMetadata": {
    "mostRecentlyExecutedCommandWithImplicitDF": {
     "commandId": 5237526414397709,
     "dataframes": [
      "_sqldf"
     ]
    },
    "pythonIndentUnit": 4
   },
   "notebookName": "Delta Live Table",
   "widgets": {}
  },
  "language_info": {
   "name": "python"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 0
}